# Reduced Data Structures
This notebook chooses a small consistent subset of the treated dataset to be run in small machines. The generated dataset is limited to a number of **drug-drug interactions** specified by the variable $N$. From this dataset, it generates the corresponding data structures like dictionaries and adjacency matrices to be fed directly to DECAGON.<br>
This code is in part the adaptation in `pandas` of the script `drug_dataset.sh`, merged with `data_structures.ipynb`.

Author: Juan Sebastian Diaz Boada, May 2020

## Python 3

In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import pickle
from joblib import Parallel, delayed

In [2]:
# Number of side effects 
#(Maximum is 1317, but the max chosen is 964 to be consistents with authors of decagon)
N = 964

In [3]:
# Here we choose specific side effects: Mumps, carbuncle, coccydynia, Bleeding,
# disorder Renal, emesis (in that order)
# ignoring the previous given number 
idx = [9,58,150,920,956,957]

### Import databases

In [78]:
PPI = pd.read_csv('small-decagon-ppi.csv',sep=',')
DTI = pd.read_csv('small-decagon-targets.csv',sep=',')
DDI = pd.read_csv('small-decagon-combo.csv',sep=',')
DSE = pd.read_csv('small-decagon-mono.csv',sep=',')
SE = pd.read_csv('../original_data/bio-decagon-effectcategories.csv',sep=',')

In [79]:
unique_genes = pd.unique(PPI[['Gene 1','Gene 2']].values.ravel())
unique_drugs = pd.unique(DDI[['STITCH 1','STITCH 2']].values.ravel())
# Number of interactions
orig_ppi = len(PPI.index)
orig_dti = len(DTI.index)
orig_ddi = len(DDI.index)
orig_dse = len(DSE.index)
# Number of nodes
orig_ddi_drugs = len(unique_drugs)
orig_ppi_genes = len(unique_genes)
orig_dti_drugs = len(pd.unique(DTI['STITCH']))
orig_dti_genes = len(pd.unique(DTI['Gene']))
orig_dse_drugs = len(pd.unique(DSE['STITCH'].values))
# Side effects
orig_se_mono = len(pd.unique(DSE['Side Effect Name']))
orig_se_combo = len(pd.unique(DDI['Polypharmacy Side Effect'].values))

### Docking database

In [80]:
DP = pd.read_csv('Docking_result_1.csv',sep=',')

In [81]:
stitch_unique = pd.unique(DP["STITCH"].values)
genes_unique = pd.unique(DP["Gene"].values)

In [82]:
print(len(stitch_unique))
print(len(genes_unique))
print(len(unique_drugs))
print(len(unique_genes))
print(len(DP))

221
723
639
19081
6162


In [83]:
stitch_unique = np.array(DP['STITCH'].value_counts().head(60).index)

In [84]:
genes_unique = np.array(DP['Gene'].value_counts().head(150).index)

In [85]:
inter_drugs = np.intersect1d(stitch_unique,unique_drugs,assume_unique=True)

In [86]:
DDI = DDI[np.logical_and(DDI['STITCH 1'].isin(inter_drugs).values,
                     DDI['STITCH 2'].isin(inter_drugs).values)]

In [96]:
unique_drugs = pd.DataFrame(pd.unique(DDI[['STITCH 1','STITCH 2']].values.ravel()))
len(unique_drugs)

60

In [97]:
np.intersect1d(genes_unique,unique_genes,assume_unique=True)
len(inter_genes)

128

In [89]:
PPI = PPI[np.logical_and(PPI['Gene 1'].isin(inter_genes).values,
                     PPI['Gene 2'].isin(inter_genes).values)]

In [99]:
unique_genes = pd.DataFrame(pd.unique(PPI[['Gene 1','Gene 2']].values.ravel()))
len(unique_genes)

128

In [104]:
unique_drugs.to_csv('drugs_for_docking.csv',header=False,index=False,sep=',')
unique_genes.to_csv('genes_for_docking.csv',header=False,index=False,sep=',')

### Choose Side effects

In [ ]:
# Sort DDI to be consistent with the authors
DDI['freq'] = DDI.groupby('Polypharmacy Side Effect')['Polypharmacy Side Effect']\
            .transform('count')
DDI = DDI.sort_values(by=['freq'], ascending=False).drop(columns=['freq'])
se = pd.unique(DDI['Polypharmacy Side Effect'].values)
se_name = pd.unique(DDI['Side Effect Name'].values)

In [ ]:
if 'idx' in locals():
    se = se[idx]
    N = len(se);
    print(N)
else:
    se = se[:N]
    print(len(se))

### Select DDIs

In [ ]:
DDI = DDI[DDI['Polypharmacy Side Effect'].isin(se)].reset_index(drop=True)
DDI_drugs = pd.unique(DDI[['STITCH 1','STITCH 2']].values.ravel()) # Unique drugs 
drug2idx = {drug: i for i, drug in enumerate(DDI_drugs)}
se_names = pd.unique(DDI['Side Effect Name']) # Unique joint side effects
se_combo_name2idx = {se: i for i, se in enumerate(se_names)}
n_drugs = len(DDI_drugs)

### Select Drug side effects

In [ ]:
DSE = DSE[DSE['STITCH'].isin(DDI_drugs)].reset_index(drop=True)
dse_drugs = len(pd.unique(DSE['STITCH'].values))
se_mono_names = pd.unique(DSE['Side Effect Name'].values) # Unique individual side effects
se_mono_name2idx = {name: i for i, name in enumerate(se_mono_names)}
n_semono = len(se_mono_names)

### Select DTIs

In [ ]:
DTI = DTI[DTI['STITCH'].isin(DDI_drugs)].reset_index(drop=True)
DTI_genes = pd.unique(DTI['Gene']) # Unique genes in DTI
DTI_drugs = pd.unique(DTI['STITCH']) # Unique drugs in DTI
dti_drugs = len(DTI_drugs)
dti_genes = len(DTI_genes)

### Select PPIs

In [ ]:
PPI = PPI[np.logical_or(PPI['Gene 1'].isin(DTI_genes),
                       PPI['Gene 2'].isin(DTI_genes))].reset_index(drop=True)
PPI_genes = pd.unique(PPI[['Gene 1','Gene 2']].values.ravel()) # Unique genes is PPI
gene2idx = {gene: i for i, gene in enumerate(PPI_genes)}
n_genes = len(PPI_genes)

### Select PFs

In [ ]:
PF = PF[PF['GeneID'].isin(PPI_genes)].reset_index(drop=True)

## Adjacency matrices and degrees

In [ ]:
# DDI adjacency matrices
def se_adj_matrix(se_name):
    m = np.zeros([n_drugs,n_drugs],dtype=int)
    seDDI = DDI[DDI['Side Effect Name'].str.match(se_name)].reset_index()
    for j in seDDI.index:
        row = drug2idx[seDDI.loc[j,'STITCH 1']]
        col = drug2idx[seDDI.loc[j,'STITCH 2']]
        m[row,col] = m[col,row] = 1
    return sp.csr_matrix(m) 
ddi_adj_list = Parallel(n_jobs=8)\
    (delayed(se_adj_matrix)(d) for d in se_combo_name2idx.keys())        
ddi_degrees_list = [np.array(drug_adj.sum(axis=0)).squeeze() for drug_adj in ddi_adj_list]

In [ ]:
# DTI adjacency matrix
dti_adj = np.zeros([n_genes,n_drugs],dtype=int)
for i in DTI.index:
    row = gene2idx[DTI.loc[i,'Gene']]
    col = drug2idx[DTI.loc[i,'STITCH']]
    dti_adj[row,col] = 1
dti_adj = sp.csr_matrix(dti_adj)

In [ ]:
# PPI adjacency matrix and degrees
ppi_adj = np.zeros([n_genes,n_genes],dtype=int)
for i in PPI.index:
    row = gene2idx[PPI.loc[i,'Gene 1']]
    col = gene2idx[PPI.loc[i,'Gene 2']]
    ppi_adj[row,col]=ppi_adj[col,row]=1
ppi_degrees = np.sum(ppi_adj,axis=0)
ppi_adj = sp.csr_matrix(ppi_adj)

In [ ]:
# Drug Feature matrix
drug_feat = np.zeros([n_drugs,n_semono],dtype=int)
for i in DSE.index:
    row = drug2idx[DSE.loc[i,'STITCH']]
    col = se_mono_name2idx[DSE.loc[i,'Side Effect Name']]
    drug_feat[row,col] = 1
drug_feat = sp.csr_matrix(drug_feat)

In [ ]:
# Protein feature matrices
prot_feat = sp.coo_matrix(
    PF[['Length', 'Mass', 'n_helices', 'n_strands', 'n_turns']].to_numpy())
norm_prot_feat = sp.coo_matrix(
    PF[['Normalized Helices(Mean)',
       'Normalized Strands(Mean)',
       'Normalized Turns(Mean)']].to_numpy())
n_pf = np.shape(prot_feat)[1]
n_npf = np.shape(norm_prot_feat)[1]

### Print

In [ ]:
# Interactions (edges)
print('Interactions (edges)')
print('Original number of PPI interactions:', orig_ppi)
print('New number of PPI interactions:', len(PPI.index))
print('\n')
print('Original number of DTI interactions:', orig_dti)
print('New number of DTI interactions:', len(DTI.index))
print('\n')
print('Original number of DDI interactions:', orig_ddi)
print('New number of DDI interactions:', len(DDI.index))
print('\n')
print('Original number of DSE interactions:', orig_dse)
print('New number of DSE interactions:', len(DSE.index))
print('\n')
# Drugs and genes (nodes)
print('Drugs and genes (nodes)')
print("Original number of drugs in DSE:",orig_dse_drugs)
print("New number of drugs in DSE:",dse_drugs)
print('\n')
print("Original number of genes in PF:",orig_pf)
print("New number of genes in PF:",len(PF.index))
print('\n')
print("Original number drugs in DTI",orig_dti_drugs)
print("New number of drugs in DTI",dti_drugs)
print('\n')
print('Original number of genes in DTI:', orig_dti_genes)
print('New number of genes in DTI:',dti_genes)
print('\n')
print('Original number of genes:',orig_ppi_genes)
print('New number of genes:', n_genes)
print('\n')
print('Original number of drugs:',orig_ddi_drugs)
print('New number of drugs:', n_drugs)
print('\n')
# Side effects
print('Side effects')
print('Original number of joint side effects:',orig_se_combo)
print('New number of joint side effects:', len(se_names))
print('\n')
print('Original number of single side effects:', orig_se_mono)
print('New number of single side effects:', n_semono)
print('\n')
# Protein side effects
print('Number of protein features:',n_pf)
print('Number of normalized protein features:',n_npf)

## Save

In [ ]:
data = {}
# Dictionaries
data['gene2idx'] = gene2idx
data['drug2idx'] = drug2idx
data['se_mono_name2idx'] = se_mono_name2idx
data['se_combo_name2idx'] = se_combo_name2idx
# DDI
data['ddi_adj_list'] = ddi_adj_list
data['ddi_degrees_list'] = ddi_degrees_list
# DTI
data['dti_adj'] = dti_adj
# PPI
data['ppi_adj'] = ppi_adj
data['ppi_degrees'] = ppi_degrees
# DSE
data['drug_feat'] = drug_feat
# PF
data['prot_feat'] = prot_feat
data['norm_prot_feat'] = norm_prot_feat

In [ ]:
filename = './data_structures/DS/DS_real_DSE_' + str(n_semono) + '_PF_' + str(n_pf) +\
'_NPF_'+ str(n_npf) + '_genes_'+str(n_genes)+'_drugs_'+str(n_drugs)+'_se_'+str(N)
print(filename)

In [ ]:
with open(filename, 'wb') as f:
    pickle.dump(data, f, protocol=3)